<a href="https://colab.research.google.com/github/Rafi076/RTFER/blob/main/ReSNet34_FTA_speed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Block 0 — Setup & Imports

In [3]:
# Block 0: Setup & Imports
import os, zipfile, random, copy
import numpy as np
from PIL import Image
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models

from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

# Reproducibility
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # set True for a bit more speed (less reproducible)

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


Block 1 — Access ZIP & Detect Folders

In [4]:
# Block 1: Access ZIP & Detect Folders (robust)
ZIP_PATH = "/content/FER-2013.zip"
ROOT     = "/content/FER-2013"   # extraction root

# 1) Unzip once
if os.path.exists(ZIP_PATH) and not os.path.isdir(ROOT):
    with zipfile.ZipFile(ZIP_PATH, "r") as z:
        z.extractall(ROOT)
    print("Unzipped FER-2013 to:", ROOT)

# 2) Find the DEEPEST folder that actually contains splits
def find_base(start):
    candidates = []
    for r, dirs, files in os.walk(start):
        if any(d in dirs for d in ["train", "test", "PublicTest", "PrivateTest"]):
            candidates.append(r)
    if not candidates:
        return start
    return sorted(candidates, key=lambda p: len(p.split("/")))[-1]

BASE = find_base(ROOT)
print("Base folder detected:", BASE)

# 3) Prefer official split names if present
TRAIN_DIR_OFF   = os.path.join(BASE, "train")
PUBLIC_DIR_OFF  = os.path.join(BASE, "PublicTest")
PRIVATE_DIR_OFF = os.path.join(BASE, "PrivateTest")

HAS_OFFICIAL = all(os.path.isdir(p) for p in [TRAIN_DIR_OFF, PUBLIC_DIR_OFF, PRIVATE_DIR_OFF])
print("Official splits found:", HAS_OFFICIAL)

# 4) Fallback (your ZIP has only train/test)
TRAIN_SPLIT = os.path.join(BASE, "train_split")
VAL_SPLIT   = os.path.join(BASE, "val_split")
TEST_DIR_FALLBACK = os.path.join(BASE, "test")

if HAS_OFFICIAL:
    TRAIN_DIR = TRAIN_DIR_OFF
    VAL_DIR   = PUBLIC_DIR_OFF
    TEST_DIR  = PRIVATE_DIR_OFF  # FINAL TEST
else:
    TRAIN_DIR = TRAIN_SPLIT if os.path.isdir(TRAIN_SPLIT) else TRAIN_DIR_OFF
    VAL_DIR   = VAL_SPLIT   if os.path.isdir(VAL_SPLIT)   else PUBLIC_DIR_OFF if os.path.isdir(PUBLIC_DIR_OFF) else TRAIN_DIR
    TEST_DIR  = TEST_DIR_FALLBACK

print("TRAIN_DIR:", TRAIN_DIR)
print("VAL_DIR:  ", VAL_DIR)
print("TEST_DIR: ", TEST_DIR)


Base folder detected: /content/FER-2013/FER-2013
Official splits found: False
TRAIN_DIR: /content/FER-2013/FER-2013/train
VAL_DIR:   /content/FER-2013/FER-2013/train
TEST_DIR:  /content/FER-2013/FER-2013/test


**Block 2 — Quick Sanity Check (counts)**

In [5]:
# Block 2: Sanity Check: class names and image counts
def count_images(root):
    total = 0
    per_class = {}
    if not os.path.isdir(root):
        return 0, {}
    for cls in sorted(os.listdir(root)):
        p = os.path.join(root, cls)
        if os.path.isdir(p):
            n = len([f for f in os.listdir(p) if f.lower().endswith(('.png','.jpg','.jpeg'))])
            per_class[cls] = n
            total += n
    return total, per_class

for name, path in [("Train", TRAIN_DIR), ("PublicTest/Val", VAL_DIR), ("PrivateTest/Final", TEST_DIR)]:
    tot, pc = count_images(path)
    print(f"{name}: total={tot}, per_class={pc}")


Train: total=28709, per_class={'angry': 3995, 'disgust': 436, 'fear': 4097, 'happy': 7215, 'neutral': 4965, 'sad': 4830, 'surprise': 3171}
PublicTest/Val: total=28709, per_class={'angry': 3995, 'disgust': 436, 'fear': 4097, 'happy': 7215, 'neutral': 4965, 'sad': 4830, 'surprise': 3171}
PrivateTest/Final: total=7178, per_class={'angry': 958, 'disgust': 111, 'fear': 1024, 'happy': 1774, 'neutral': 1233, 'sad': 1247, 'surprise': 831}


**Block 3 — Transforms (CLAHE + augs) & Eval (TenCrop + Fast CenterCrop)**

In [6]:
# Block 3: Transforms
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

# CLAHE -> returns a PIL RGB image
class CLAHE_PIL(object):
    def __init__(self, clip=2.0, grid=(8,8)):
        self.clip = clip; self.grid = grid
    def __call__(self, img: Image.Image):
        g = np.array(img.convert("L"))
        clahe = cv2.createCLAHE(clipLimit=self.clip, tileGridSize=self.grid)
        g = clahe.apply(g)
        rgb = cv2.cvtColor(g, cv2.COLOR_GRAY2RGB)
        return Image.fromarray(rgb)

# Train pipeline: CLAHE + strong augs
train_transform = transforms.Compose([
    CLAHE_PIL(clip=2.0, grid=(8,8)),
    transforms.Resize(56),
    transforms.RandomResizedCrop(48, scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# Eval pipeline: TenCrop (TTA)
eval_transform = transforms.Compose([
    CLAHE_PIL(clip=2.0, grid=(8,8)),
    transforms.Resize(56),
    transforms.TenCrop(48),
    transforms.Lambda(lambda crops: torch.stack([
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)(
            transforms.ToTensor()(c)
        ) for c in crops
    ])),
])

# FAST eval pipeline: single CenterCrop for quick validation
eval_transform_fast = transforms.Compose([
    CLAHE_PIL(clip=2.0, grid=(8,8)),
    transforms.Resize(56),
    transforms.CenterCrop(48),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])


**Block 4 — Datasets & Dataloaders (class-map fixed + fast val loader)**

In [7]:
# Block 4: Datasets & Dataloaders (fixed class mapping + fast val loader)
assert os.path.isdir(TRAIN_DIR), f"Missing TRAIN_DIR: {TRAIN_DIR}"
assert os.path.isdir(TEST_DIR),  f"Missing TEST_DIR: {TEST_DIR}"
assert os.path.isdir(VAL_DIR),   f"Missing VAL_DIR: {VAL_DIR}"

train_ds      = datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
val_ds        = datasets.ImageFolder(VAL_DIR,   transform=eval_transform)
val_ds_fast   = datasets.ImageFolder(VAL_DIR,   transform=eval_transform_fast)
test_ds       = datasets.ImageFolder(TEST_DIR,  transform=eval_transform)

# ---- Force consistent class mapping using TRAIN classes ----
fixed_classes = train_ds.classes
fixed_map = {cls: i for i, cls in enumerate(fixed_classes)}

def remap_dataset_targets(ds, fixed_map):
    local_classes = ds.classes
    idx2global = {i: fixed_map[c] for i, c in enumerate(local_classes) if c in fixed_map}

    if hasattr(ds, "samples"):
        new_samples = []
        for p, t in ds.samples:
            if t in idx2global:
                new_samples.append((p, idx2global[t]))
        ds.samples = new_samples
        ds.targets = [t for _, t in ds.samples]

    ds.classes = list(fixed_map.keys())
    ds.class_to_idx = dict(fixed_map)
    return ds

val_ds      = remap_dataset_targets(val_ds, fixed_map)
val_ds_fast = remap_dataset_targets(val_ds_fast, fixed_map)
test_ds     = remap_dataset_targets(test_ds, fixed_map)

print("Fixed classes:", fixed_classes)
print("Train n:", len(train_ds), "| Val n:", len(val_ds), "| Test n:", len(test_ds))

# Dataloader helper (tries speed flags and falls back if unsupported)
def make_loader(ds, batch_size, shuffle):
    try:
        return DataLoader(ds, batch_size=batch_size, shuffle=shuffle,
                          num_workers=2, pin_memory=True,
                          persistent_workers=True, prefetch_factor=4)
    except TypeError:
        return DataLoader(ds, batch_size=batch_size, shuffle=shuffle,
                          num_workers=2, pin_memory=True)

BATCH_TRAIN = 64
BATCH_EVAL  = 64  # a bit larger since eval is lighter

train_loader    = make_loader(train_ds,    BATCH_TRAIN, True)
val_loader      = make_loader(val_ds,      BATCH_EVAL,  False)      # TenCrop (5D)
val_loader_fast = make_loader(val_ds_fast, BATCH_EVAL,  False)      # CenterCrop (4D)
test_loader     = make_loader(test_ds,     BATCH_EVAL,  False)      # TenCrop (5D)


Fixed classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Train n: 28709 | Val n: 28709 | Test n: 7178


**Block 5 — Model (ResNet34) + Loss/Opt/Scheduler**

In [8]:
# Block 5: Model + Loss/Opt/Scheduler
class EmotionResNet34(nn.Module):
    def __init__(self, num_classes=7, dropout=0.5):
        super().__init__()
        self.backbone = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Dropout(dropout),
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        self._init_weights()
    def _init_weights(self):
        for m in self.classifier:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    def forward(self, x):
        feat = self.backbone(x)
        out  = self.classifier(feat)
        return out

model = EmotionResNet34().to(device)

# Label smoothing loss
class LabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing
    def forward(self, pred, target):
        log_probs = F.log_softmax(pred, dim=-1)
        n = pred.size(1)
        true_dist = torch.zeros_like(log_probs)
        true_dist.scatter_(1, target.unsqueeze(1), self.confidence)
        true_dist += self.smoothing / n
        return torch.mean(torch.sum(-true_dist * log_probs, dim=-1))

criterion = LabelSmoothingLoss(0.1)
optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
scaler    = torch.cuda.amp.GradScaler()


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


100%|██████████| 83.3M/83.3M [00:00<00:00, 198MB/s]
/tmp/ipython-input-2156613188.py:52: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler    = torch.cuda.amp.GradScaler()


**Block 6 — Evaluation Utils (TenCrop + Fast CenterCrop)**

In [9]:
# Block 6: Evaluation Utils

@torch.no_grad()
def evaluate_tencrop(model, loader):
    """Eval with TenCrop TTA; loader yields 5D tensors: [B, 10, C, H, W]."""
    model.eval()
    total, correct = 0, 0
    for images, labels in loader:
        bs, ncrops, c, h, w = images.size()
        images = images.view(-1, c, h, w).to(device)
        labels = labels.to(device)
        with torch.autocast(device_type='cuda', enabled=(device.type=='cuda')):
            logits = model(images)              # [B*10, 7]
        logits = logits.view(bs, ncrops, -1).mean(1)  # avg over 10 crops
        preds = logits.argmax(1)
        correct += (preds == labels).sum().item()
        total   += labels.size(0)
    return correct / total

@torch.no_grad()
def evaluate_center(model, loader):
    """Fast eval with single CenterCrop; loader yields 4D tensors: [B, C, H, W]."""
    model.eval()
    total, correct = 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        with torch.autocast(device_type='cuda', enabled=(device.type=='cuda')):
            logits = model(images)
        preds = logits.argmax(1)
        correct += (preds == labels).sum().item()
        total   += labels.size(0)
    return correct / total


**Block 7 — Stage-1 Training (freeze) — FAST**

In [10]:
# Block 7 (FAST): Stage-1 Training (Freeze backbone)
for p in model.backbone.parameters():
    p.requires_grad = False

best_val_acc_tta  = 0.0   # best TenCrop val acc (we save on this)
best_val_acc_fast = 0.0   # track fast proxy
best_wts = copy.deepcopy(model.state_dict())

EPOCHS_S1, PATIENCE_S1 = 5, 2

for epoch in range(EPOCHS_S1):
    model.train()
    running_loss = 0.0; correct = 0; total = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.autocast(device_type='cuda', enabled=(device.type=='cuda')):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * x.size(0)
        correct      += (logits.argmax(1) == y).sum().item()
        total        += y.size(0)

    scheduler.step()
    train_loss = running_loss / total
    train_acc  = correct / total

    # FAST validation (CenterCrop)
    val_acc_fast = evaluate_center(model, val_loader_fast)
    print(f"[S1][{epoch+1}/{EPOCHS_S1}] loss={train_loss:.4f} | train_acc={train_acc:.4f} | val_acc_fast={val_acc_fast:.4f}")

    # If fast validation improved, confirm with TenCrop (expensive)
    improved = val_acc_fast > best_val_acc_fast + 1e-6
    if improved:
        best_val_acc_fast = val_acc_fast
        val_acc_tta = evaluate_tencrop(model, val_loader)
        print(f"   ↳ TenCrop-check: {val_acc_tta:.4f}")
        if val_acc_tta > best_val_acc_tta + 1e-6:
            best_val_acc_tta = val_acc_tta
            best_wts = copy.deepcopy(model.state_dict())
            torch.save(best_wts, "/content/best_fer_model_stage1.pth")
            print("   ↳ Saved best Stage-1 weights.")
            PATIENCE_S1 = 2  # reset a bit
        else:
            PATIENCE_S1 -= 1
    else:
        PATIENCE_S1 -= 1

    if PATIENCE_S1 <= 0:
        print("   ↳ Early stop Stage-1.")
        break

# Load best Stage-1
model.load_state_dict(best_wts)


[S1][1/5] loss=2.1507 | train_acc=0.2137 | val_acc_fast=0.2963
   ↳ TenCrop-check: 0.3121
   ↳ Saved best Stage-1 weights.
[S1][2/5] loss=1.8638 | train_acc=0.2513 | val_acc_fast=0.3049
   ↳ TenCrop-check: 0.3061
[S1][3/5] loss=1.8221 | train_acc=0.2671 | val_acc_fast=0.3165
   ↳ TenCrop-check: 0.3281
   ↳ Saved best Stage-1 weights.
[S1][4/5] loss=1.8074 | train_acc=0.2770 | val_acc_fast=0.3177
   ↳ TenCrop-check: 0.3323
   ↳ Saved best Stage-1 weights.
[S1][5/5] loss=1.8015 | train_acc=0.2851 | val_acc_fast=0.3229
   ↳ TenCrop-check: 0.3418
   ↳ Saved best Stage-1 weights.


<All keys matched successfully>

**Block 8 — Stage-2 Fine-Tuning (unfreeze layer4 only; discrim LRs) — FAST**

In [11]:
# Block 8 (FAST): Stage-2 Fine-Tune (unfreeze only layer4; discriminative LRs)
for name, p in model.backbone.named_parameters():
    p.requires_grad = ("layer4" in name)  # only last block learns

# Two param groups: lower LR for backbone layer4, higher LR for head
head_params = list(model.classifier.parameters())
l4_params   = [p for n,p in model.backbone.named_parameters() if "layer4" in n]
optimizer = AdamW([
    {"params": l4_params,   "lr": 1e-4},
    {"params": head_params, "lr": 3e-4},
], weight_decay=1e-4)

scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=8, T_mult=2)

best_val_acc_ft_tta  = best_val_acc_tta
best_val_acc_ft_fast = best_val_acc_fast
best_wts_ft = copy.deepcopy(model.state_dict())

EPOCHS_S2, PATIENCE_S2 = 12, 3

for epoch in range(EPOCHS_S2):
    model.train()
    running_loss = 0.0; correct = 0; total = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.autocast(device_type='cuda', enabled=(device.type=='cuda')):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * x.size(0)
        correct      += (logits.argmax(1) == y).sum().item()
        total        += y.size(0)

    scheduler.step()
    train_loss = running_loss / total
    train_acc  = correct / total

    val_acc_fast = evaluate_center(model, val_loader_fast)
    print(f"[S2][{epoch+1}/{EPOCHS_S2}] loss={train_loss:.4f} | train_acc={train_acc:.4f} | val_acc_fast={val_acc_fast:.4f}")

    improved = val_acc_fast > best_val_acc_ft_fast + 1e-6
    if improved:
        best_val_acc_ft_fast = val_acc_fast
        val_acc_tta = evaluate_tencrop(model, val_loader)
        print(f"   ↳ TenCrop-check: {val_acc_tta:.4f}")
        if val_acc_tta > best_val_acc_ft_tta + 1e-6:
            best_val_acc_ft_tta = val_acc_tta
            best_wts_ft = copy.deepcopy(model.state_dict())
            torch.save(best_wts_ft, "/content/best_fer_model_finetuned.pth")
            print("   ↳ Saved best Stage-2 weights.")
            PATIENCE_S2 = 3
        else:
            PATIENCE_S2 -= 1
    else:
        PATIENCE_S2 -= 1

    if PATIENCE_S2 <= 0:
        print("   ↳ Early stop Stage-2.")
        break

# Load best fine-tuned model
model.load_state_dict(best_wts_ft)


[S2][1/12] loss=1.7310 | train_acc=0.3369 | val_acc_fast=0.4164
   ↳ TenCrop-check: 0.4531
   ↳ Saved best Stage-2 weights.
[S2][2/12] loss=1.6559 | train_acc=0.3873 | val_acc_fast=0.4477
   ↳ TenCrop-check: 0.4803
   ↳ Saved best Stage-2 weights.
[S2][3/12] loss=1.6170 | train_acc=0.4132 | val_acc_fast=0.4647
   ↳ TenCrop-check: 0.5003
   ↳ Saved best Stage-2 weights.
[S2][4/12] loss=1.5907 | train_acc=0.4291 | val_acc_fast=0.4818
   ↳ TenCrop-check: 0.5179
   ↳ Saved best Stage-2 weights.
[S2][5/12] loss=1.5736 | train_acc=0.4325 | val_acc_fast=0.4876
   ↳ TenCrop-check: 0.5186
   ↳ Saved best Stage-2 weights.
[S2][6/12] loss=1.5562 | train_acc=0.4422 | val_acc_fast=0.4958
   ↳ TenCrop-check: 0.5342
   ↳ Saved best Stage-2 weights.
[S2][7/12] loss=1.5448 | train_acc=0.4507 | val_acc_fast=0.4979
   ↳ TenCrop-check: 0.5368
   ↳ Saved best Stage-2 weights.
[S2][8/12] loss=1.5369 | train_acc=0.4548 | val_acc_fast=0.4989
   ↳ TenCrop-check: 0.5386
   ↳ Saved best Stage-2 weights.
[S2][9/1

<All keys matched successfully>

**Block 9 — FINAL Test (TenCrop TTA) on test**

In [12]:
# Block 9: FINAL Test (PrivateTest if available)
final_ckpt = "/content/best_fer_model_finetuned.pth"
if not os.path.exists(final_ckpt):
    final_ckpt = "/content/best_fer_model_stage1.pth"

if os.path.exists(final_ckpt):
    model.load_state_dict(torch.load(final_ckpt, map_location=device))
    print("Loaded best checkpoint:", final_ckpt)
else:
    print("No saved checkpoint found; using in-memory weights.")

final_test_acc = evaluate_tencrop(model, test_loader)
split_name = "PrivateTest" if os.path.basename(TEST_DIR).lower() == "privatetest" else "test/"
print(f"\n FINAL TEST ACCURACY (TenCrop TTA) on {split_name}: {final_test_acc*100:.2f}%")


Loaded best checkpoint: /content/best_fer_model_finetuned.pth

 FINAL TEST ACCURACY (TenCrop TTA) on test/: 52.54%


**Block 10 — Confusion Matrix on Final Test**

In [13]:
# Block 10: Confusion Matrix (optional)
from sklearn.metrics import confusion_matrix, classification_report

@torch.no_grad()
def predict_all(model, loader):
    model.eval(); ys, yh = [], []
    for images, labels in loader:
        bs, ncrops, c, h, w = images.size()
        images = images.view(-1, c, h, w).to(device)
        labels = labels.to(device)
        with torch.autocast(device_type='cuda', enabled=(device.type=='cuda')):
            logits = model(images)
        logits = logits.view(bs, ncrops, -1).mean(1)
        preds = logits.argmax(1)
        ys.append(labels.cpu().numpy())
        yh.append(preds.cpu().numpy())
    return np.concatenate(ys), np.concatenate(yh)

y_true, y_pred = predict_all(model, test_loader)
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=fixed_classes))


Confusion Matrix:
 [[ 396    0   32  155  131  214   30]
 [  57    0    4   17    6   22    5]
 [ 152    0  125  136  174  304  133]
 [  54    0   16 1466   86  122   30]
 [  77    0   36  220  639  220   41]
 [ 152    0   59  181  232  599   24]
 [  43    0   51   77   78   36  546]]

Classification Report:
               precision    recall  f1-score   support

       angry       0.43      0.41      0.42       958
     disgust       0.00      0.00      0.00       111
        fear       0.39      0.12      0.19      1024
       happy       0.65      0.83      0.73      1774
     neutral       0.47      0.52      0.50      1233
         sad       0.39      0.48      0.43      1247
    surprise       0.67      0.66      0.67       831

    accuracy                           0.53      7178
   macro avg       0.43      0.43      0.42      7178
weighted avg       0.50      0.53      0.50      7178



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
